
# Práctica 4: Diagnóstico de cáncer con genes

## Objetivo
El fichero “genes.csv” contiene información de dos genes y la posibilidad de tener cáncer o no. Con todo ello
se quiere crear un modelo que permita hacer un diagnóstico para un nuevo paciente teniendo en cuenta su
similitud en un campo de n-dimensiones.


Elige el clasificador que más se adapte de entre los vistos en clase y usa scikit-learn junto con las librerías que
necesites para resolver las siguientes cuestiones.

Importamos las librerías necesarias y guardamos los datos en una variable:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import math

In [ ]:
data = pd.read_csv('genes.csv', delimiter=None, header=0)

data

,Gene1,Gene2,CancerPresent
0,4.3,3.9,1
1,2.5,6.3,0
2,5.7,3.9,1
3,6.1,6.2,0
4,7.4,3.4,1
...,...,...,...
2995,5.0,6.5,1
2996,3.4,6.6,0
2997,2.7,6.5,0
2998,3.3,5.6,0


Para este tipo de problema emplearemos el algoritmo **Vecinos más cercanos**.

Para empezar veremos si hay alguna columna con valores NaN:

In [ ]:
data.isnull().sum()

Gene1            0
Gene2            0
CancerPresent    0
dtype: int64

En este caso todas las columas están rellenas, por lo que pasaremos al siguiente paso: eliminaremos las filas duplicadas en caso de que hubiera ya que influye en la distancia entre los puntos. Si los datos duplicados están presentes, el algoritmo considerará esos puntos duplicados como vecinos cercanos y puede sesgar los resultados:

In [ ]:
data = data.drop_duplicates()
data

,Gene1,Gene2,CancerPresent
0,4.3,3.9,1
1,2.5,6.3,0
2,5.7,3.9,1
3,6.1,6.2,0
4,7.4,3.4,1
...,...,...,...
2995,5.0,6.5,1
2996,3.4,6.6,0
2997,2.7,6.5,0
2998,3.3,5.6,0


A continuación, dividiremos los sets de entrenamiento, validación y test:

In [ ]:
# Primero separamos las características y sus etiquetas
X = data.iloc[:, :-1]  # Características
y = data.iloc[:, -1]   # Etiquetas

# Dividimos los datos en conjuntos de entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalizamos las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

***Primer modelo***

El primer modelo que crearemos será con 5 vecinos y con la distancia de 'Minkowski'.

Creamos el modelo con la función `KNeighborsClassifier`.

In [ ]:
algoritmo = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

#Entreno el modelo
algoritmo.fit(X_train,y_train)

y_pred = algoritmo.predict(X_test)

Una vez que hemos entrenado el primer modelo, pintaremos la matriz de confusión que nos permitirá ver su calidad y precisión.

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[150  14]
 [ 12 145]]


Esto quiere decir que contamos con 150 verdaderos positivos y con 14 falsos positivos. También contamos con 12 falsos negativos y 145 verdaderos negativos.

In [ ]:
precision = precision_score(y_test, y_pred,average='micro')
print('Precisión:')
print(precision)

Precisión:
0.9190031152647975


Se trata de un buen modelo ya que cuenta con una precisión del 91,9%.

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       164
           1       0.91      0.92      0.92       157

    accuracy                           0.92       321
   macro avg       0.92      0.92      0.92       321
weighted avg       0.92      0.92      0.92       321



Observamos que la clase que mejor predice es la clase 0 con un 93% mientras que la clase 1 la predice con una precisión del 91%, ambos porcentajes altos.

In [ ]:
tabla = pd.DataFrame()
tabla.insert(0, "Valor Real",list(y_test), True)
tabla.insert(1, "Valor_Predicho mod 1",list(y_pred), True)
tabla

,Valor Real,Valor_Predicho mod 1
0,0,0
1,1,1
2,1,1
3,0,0
4,0,1
...,...,...
316,0,0
317,1,1
318,0,0
319,1,1


**Segundo modelo**

Para este segundo modelo utilizaremos 8 vecinos, para ver qué ocurre con un número par, y la misma distancia.

In [ ]:
algoritmo2 = KNeighborsClassifier(n_neighbors = 8, metric = 'minkowski', p = 2)
algoritmo2.fit(X_train, y_train)
y_pred2 = algoritmo2.predict(X_test)
matriz2 = confusion_matrix(y_test, y_pred2)
print('Matriz de Confusión:')
print(matriz2)

Matriz de Confusión:
[[154  10]
 [ 18 139]]


Esto quiere decir que contamos con 154 verdaderos positivos y con 10 falsos positivos. También contamos con 18 falsos negativos y 139 verdaderos negativos.

In [ ]:
precision2 = precision_score(y_test, y_pred2,average='micro')
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.9127725856697819


Se trata de un buen modelo ya que cuenta con una precisión del 91,28%, sin embargo es un poco peor que el primer modelo.

In [ ]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       164
           1       0.93      0.89      0.91       157

    accuracy                           0.91       321
   macro avg       0.91      0.91      0.91       321
weighted avg       0.91      0.91      0.91       321



En este caso, al contrario que en el anterior modelo, la clase 1, que cuenta con un 93% de precisión, se predice mejor que la clase 0 que cuenta con un 90% de precisión.

In [ ]:
tabla.insert(2, "Valor_Predicho mod 2",list(y_pred2), True)
tabla

,Valor Real,Valor_Predicho mod 1,Valor_Predicho mod 2
0,0,0,0
1,1,1,1
2,1,1,1
3,0,0,0
4,0,1,1
...,...,...,...
316,0,0,0
317,1,1,1
318,0,0,0
319,1,1,1


**Tercer modelo**

Como hemos visto que 5 vecinos son mejores que 8, cambiemos ahora la distancia por la euclídea.

In [ ]:
algoritmo3 = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
algoritmo3.fit(X_train, y_train)
y_pred3 = algoritmo3.predict(X_test)
matriz3 = confusion_matrix(y_test, y_pred3)
print('Matriz de Confusión:')
print(matriz3)

Matriz de Confusión:
[[150  14]
 [ 12 145]]


Con la distancia euclídea obtenemos los mismos valores que hemos obtenido con la distancia Minkowski.

In [ ]:
precision3 = precision_score(y_test, y_pred3,average='micro')
print('Precisión:')
print(precision3)

Precisión:
0.9190031152647975


In [ ]:
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       164
           1       0.91      0.92      0.92       157

    accuracy                           0.92       321
   macro avg       0.92      0.92      0.92       321
weighted avg       0.92      0.92      0.92       321



Comprobamos que hemos obtenidos los mismos resultados.

In [ ]:
tabla.insert(3, "Valor_Predicho mod 3",list(y_pred3), True)
tabla

,Valor Real,Valor_Predicho mod 1,Valor_Predicho mod 2,Valor_Predicho mod 3
0,0,0,0,0
1,1,1,1,1
2,1,1,1,1
3,0,0,0,0
4,0,1,1,1
...,...,...,...,...
316,0,0,0,0
317,1,1,1,1
318,0,0,0,0
319,1,1,1,1


**Cuarto Modelo**

Veamos ahora qué sucede si aumentamos más los vecinos pero mantiendo la distancia en euclídea.

In [ ]:
algoritmo4 = KNeighborsClassifier(n_neighbors = 11, metric = 'euclidean')
algoritmo4.fit(X_train, y_train)
y_pred4 = algoritmo4.predict(X_test)
matriz4= confusion_matrix(y_test, y_pred4)
print('Matriz de Confusión:')
print(matriz4)

Matriz de Confusión:
[[152  12]
 [ 12 145]]


En este caso contamos con 152 verdaderos positivos y con 12 falsos positivos. También contamos con 12 falsos negativos y 145 verdaderos negativos.

In [ ]:
precision4 = precision_score(y_test, y_pred4,average='micro')
print('Precisión:')
print(precision4)

Precisión:
0.9252336448598131


Obtenemos una precisión del 92,52% siendo el mejor modelo obtenido hasta ahora.

In [ ]:
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       164
           1       0.92      0.92      0.92       157

    accuracy                           0.93       321
   macro avg       0.93      0.93      0.93       321
weighted avg       0.93      0.93      0.93       321



Contamos con una precisión del 92% para la clase 1 y del 93% para la clase 0.

In [ ]:
tabla.insert(4, "Valor_Predicho mod 4",list(y_pred4), True)
tabla

,Valor Real,Valor_Predicho mod 1,Valor_Predicho mod 2,Valor_Predicho mod 3,Valor_Predicho mod 4
0,0,0,0,0,0
1,1,1,1,1,1
2,1,1,1,1,1
3,0,0,0,0,0
4,0,1,1,1,1
...,...,...,...,...,...
316,0,0,0,0,0
317,1,1,1,1,1
318,0,0,0,0,0
319,1,1,1,1,1


Finalmente trabajaremos con el modelo 4 que emplea 11 vecinos y la distancia euclídea para ver qué ocurre con los datos de validación.

In [ ]:
prediccion = algoritmo4.predict(X_val)

matriz_conf= confusion_matrix(y_val, prediccion)
print('Matriz de Confusión:')
print(matriz_conf)

Matriz de Confusión:
[[134  13]
 [ 11 163]]


En este caso contamos con 134 verdaderos positivos y con 13 falsos positivos. También contamos con 11 falsos negativos y 163 verdaderos negativos.

In [ ]:
Precision = precision_score(y_val, prediccion,average='micro')
print('Precisión:')
print(Precision)

Precisión:
0.9252336448598131


Seguimos contando con una precisión del 92,52%.

In [ ]:
print(classification_report(y_val,prediccion))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92       147
           1       0.93      0.94      0.93       174

    accuracy                           0.93       321
   macro avg       0.93      0.92      0.92       321
weighted avg       0.93      0.93      0.93       321



Contamos con una precisión del 92% para la clase 0 y del 93% para la clase 1.

In [ ]:
tablaval = pd.DataFrame()
tablaval.insert(0, "Valor Real",list(y_val), True)
tablaval.insert(1, "Valor Predicho",list(prediccion), True)
tablaval

,Valor Real,Valor Predicho
0,0,0
1,1,1
2,1,1
3,0,0
4,1,1
...,...,...
316,0,0
317,1,1
318,1,1
319,1,1
